In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm
from imaris_ims_file_reader.ims import ims
import dask.array as da
import pandas as pd
import numpy as np
import pickle as pkl
import napari
from cellpose import models
from skimage.io import imread,imsave
import seaborn as sns
import matplotlib.pyplot as plt

from utils import extract_regionprops, find_edge_df, mask_from_df, suppress_by_iou_mitos, compute_iou_array, add_soma_data

In [19]:
im_path = r'D:\data_analysis\2025_Birder_mito\88EM87C 25x25_ashlar.ome.tif'
output_dir = r'D:\data_analysis\2025_Birder_mito\C_00_analysis'
prefix_save = '88EM87C_'
mitos_sub_dir = 'mitos_sam'

px_size = 10 #[nm]

df_axons_path = os.path.join(output_dir,f'{prefix_save}axons.pkl')
df_mitos_path = os.path.join(output_dir,f'{prefix_save}mitos.pkl')

axon_res = 3
mitos_res = 0
row_offset = 0 # used if df was created as a test on a smaller image
col_offset = 0 # used if df was created as a test on a smaller image

# details of selecting mito masks
res_adjust = axon_res - mitos_res

## Get an image

In [9]:
store = imread(im_path, aszarr=True)
im_list = []
for res in range(axon_res+1):
    im = da.from_zarr(store,res)
    im_list.append(im)

C:\Users\KMK280\AppData\Local\Temp\ipykernel_38664\1588550211.py:1: FutureWarning: The plugin infrastructure in `skimage.io` is deprecated since version 0.25 and will be removed in 0.27 (or later). To avoid this warning, please do not pass additional keyword arguments for plugins (`**plugin_args`). Instead, use `imageio` or other I/O packages directly. See also `skimage.io.imread`.
  store = imread(im_path, aszarr=True)


In [10]:
im_shape = im_list[0].shape
im_shape

(22084, 29266)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(im_list, scale = [px_size, px_size])

<Image layer 'im_list' at 0x1b788040090>

In [25]:
viewer.scale_bar.visible = True

# Text options
viewer.scale_bar.unit = 'nm'  # set to None to diplay no unit
viewer.scale_bar.length = 5000  # length, in units, of the scale bar
viewer.scale_bar.font_size = 20  # default is 10

# Text color
viewer.scale_bar.colored = True  # default value is False
viewer.scale_bar.color = 'white'  # default value is magenta: (1,0,1,1)

# Background box
viewer.scale_bar.box = False  # add background box, default is False

# Scale bar position
viewer.scale_bar.position = 'bottom_right'  # default is 'bottom_right'
viewer.scale_bar.ticks = False

## Visualize axons

In [14]:
df_name = f'{prefix_save}axons.pkl'
df_path = os.path.join(output_dir,df_name)
df_cells = pd.read_pickle(df_path)
print('Number of axons:',len(df_cells))

Number of axons: 548


In [26]:
im_shape = im_list[axon_res].shape

# add mask to viewer
mask = mask_from_df(df_cells, im_shape)
viewer.add_labels(mask, scale = [px_size*2**axon_res,px_size*2**axon_res])

<Labels layer 'mask' at 0x1b796cba350>

## Visualize mitochondria

In [15]:
df_name = f'{prefix_save}mitos.pkl'
df_path = os.path.join(output_dir,df_name)
df_mitos = pd.read_pickle(df_path)
print('Number of axons:',len(df_mitos))

Number of axons: 4347


In [27]:
# add points for each category
categories = ['healthy', 'swollen']
colors = ['green', 'red']

for cat, color in zip(categories, colors):

    points = df_mitos.loc[df_mitos.prediction == cat,['centroid-0','centroid-1']].to_numpy()
    
    viewer.add_points(points, name = cat, size = 50, border_color= color, face_color = [0, 0, 0, 0], scale = [px_size*2**mitos_res,px_size*2**mitos_res])